In [1]:
import pandas as pd
import numpy as np
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

cred = credentials.Certificate("thecrowsnestapp-creds.json")
firebase_admin.initialize_app(cred)

db = firestore.client()

In [63]:
from openskill.models import PlackettLuce, BradleyTerryFull
model = PlackettLuce()

In [37]:
%load_ext scalene

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version. To profile in line mode, use
`%scrun [options] statement`. To profile in cell mode, use `%%scalene
[options]` followed by your code.

NOTE: in Jupyter notebook on MacOS, Scalene cannot profile child
processes. Do not run to try Scalene with multiprocessing in Jupyter
Notebook.


In [165]:
baseElo = 1000
baseSigma = baseElo // 3

In [166]:
class Sailor:
    def __init__(self, name, year, link, teams, pos, seasons=[], rank=0, rating=baseElo, races=[]):
        self.name = name
        self.year = year
        self.link = link
        self.teams = teams
        self.pos = pos
        self.rank = rank
        self.seasons = seasons
        self.races = []
        self.rivals = {}
        self.r = model.rating(rating, rating / 3, name)
        self.avgRatio = 0
        
    def rerate(self, rating):
        self.r = model.rating(rating.mu, rating.sigma, self.name)
        
    def __repr__(self):
        return f"{self.name}: {self.teams}, {self.pos} {str(self.r)} {self.races}"

In [167]:
class Team:
    def __init__(self, name, link, members=[]):
        self.name = name
        self.link = link
        self.members = members

In [168]:
def adjust_race_id(row):
    if row['Scoring'] == 'Combined':
        return row['raceID'][:-1]  # Remove the last character (A/B) for combined scoring
    return row['raceID']

In [ ]:
p1 = Sailor("p1", ['nu'], 'Skipper')
p2 = Sailor("p2", ['nu'], 'Skipper')
p3 = Sailor("p3", ['nu'], 'Skipper')
p4 = Sailor("p4", ['nu'], 'Skipper')
players = [p1,p2,p3,p4]
ratings = [[p.r] for p in players]
ratings = model.rate(ratings, [3,2,1,4], weights=[[3.0]] * 4)
for p,n in zip(players, ratings):
    p.r = n[0]
print(p1,p2,p3,p4)

In [169]:
df_races = pd.read_json("races_new.json")
# converters={"Teams": lambda x: [y.strip().split("'")[1] for y in x.strip("[]").split(", ")]}
df_races['raceNum'] = df_races['raceID'].apply(lambda id: int(id.split("/")[2][:-1]))  # Numeric part
df_races['raceDiv'] = df_races['raceID'].apply(lambda id: id.split("/")[2][-1])  # Division part (e.g., 'A', 'B')
df_races['adjusted_raceID'] = df_races.apply(adjust_race_id, axis=1) # to make combined division combined
# df_races = df_races.loc[df_races['raceID'].apply(lambda id: id.split("/")[0] == 'f24')]

df_races_full = df_races.sort_values(['Date', 'raceNum', 'raceDiv']).reset_index(drop=True)

df_races_skipper = df_races_full.loc[df_races_full['Position'].str.contains('Skipper')].sort_values(['Date', 'raceNum']).reset_index(drop=True) # filter for skippers
df_races_crew = df_races_full.loc[df_races_full['Position'].str.contains('Crew')].sort_values(['Date', 'raceNum']).reset_index(drop=True) # filter for skippers


In [170]:
teamRegions = {'Hawaii': 'PCCSC', 'Brown': 'NEISA', 'Southern Cal': 'PCCSC', 'Salve Regina': 'NEISA', 'UC Santa Barbara': 'PCCSC', 'Cal Poly': 'PCCSC', 'Washington': 'NWICSA', 'Channel Islands': 'PCCSC', 'UC San Diego': 'PCCSC', 'British Columbia': 'NWICSA', 'UC Los Angeles': 'PCCSC', 'Westmont College': 'PCCSC', 'Arizona State': 'PCCSC', 'Texas A&M Galveston': 'SEISA', 'Texas A&M': 'SEISA', 'Tulane': 'SEISA', 'Rice': 'SEISA', 'Texas': 'SEISA', 'Oklahoma State': 'SEISA', 'Texas A&M C. Christ': 'SEISA', 'Central Oklahoma': 'SEISA', 'Notre Dame': 'MCSA', 'Jacksonville': 'SAISA', 'Florida': 'SAISA', 'Tennessee': 'SAISA', 'Rollins': 'SAISA', 'North Carolina State': 'SAISA', 'Georgia Tech': 'SAISA', 'Auburn': 'SAISA', 'Charleston': 'SAISA', 'South Florida': 'SAISA', 'Old Dominion': 'MAISA', 'Eckerd': 'SAISA', 'Florida State': 'SAISA', 'U. Miami': 'SAISA', 'UW Milwaukee': 'MCSA', 'Stony Brook': 'MAISA', 'Duke': 'SAISA', 'Clemson': 'SAISA', 'U South Carolina': 'SAISA', 'UNC Wilmington': 'SAISA', 'Georgia': 'SAISA', 'Berkeley': 'PCCSC', 'CSU Long Beach': 'PCCSC', 'Monterey Bay': 'PCCSC', 'UC Irvine': 'PCCSC', 'UC Davis': 'PCCSC', 'Rhode Island': 'NEISA', 'Georgetown': 'MAISA', 'Dartmouth': 'NEISA', 'MIT': 'NEISA', 'George Washington': 'MAISA', 'Navy': 'MAISA', 'Fordham': 'MAISA', 'Northeastern': 'NEISA', 'Christopher Newport': 'MAISA', 'Victoria': 'NWICSA', 'Boston University': 'NEISA', 'Miami University': 'MCSA', 'Hampton': 'MAISA', 'Virginia': 'MAISA', 'Stevens': 'MAISA', 'Columbia': 'MAISA', 'NY Maritime': 'MAISA', 'Kings Point': 'MAISA', "St. Mary's": 'MAISA', 'Maryland': 'MAISA', 'Virginia Tech': 'MAISA', 'Drexel': 'MAISA', 'Maryland/Baltimore': 'MAISA', 'Buffalo': 'MAISA', 'UC Santa Cruz': 'PCCSC', 'Santa Clara': 'PCCSC', 'Wisconsin': 'MCSA', 'Michigan': 'MCSA', 'Washington College': 'MAISA', 'Minnesota': 'MCSA', 'Yale': 'NEISA', 'Hobart & William': 'MAISA', 'Vermont': 'NEISA', 'Connecticut College': 'NEISA', 'Harvard': 'NEISA', 'Roger Williams': 'NEISA', 'Syracuse': 'MAISA', 'Tufts': 'NEISA', 'Middlebury': 'NEISA', 'New College': 'SAISA', 'William and Mary': 'MAISA', 'Gannon': 'MAISA', 'Boston College': 'NEISA', 'Stanford': 'PCCSC', 'Bowdoin': 'NEISA', 'Lewis & Clark': 'NWICSA', 'Monmouth': 'MAISA', 'American': 'MAISA', 'Michigan State': 'MCSA', 'Hope': 'MCSA', 'Western Michigan': 'MCSA', 'Toledo': 'MCSA', 'Ohio State': 'MCSA', 'Mass Maritime': 'NEISA', 'Coast Guard': 'NEISA', 'Bates': 'NEISA', 'Fairfield': 'NEISA', 'Sacred Heart': 'NEISA', 'Wentworth Institute': 'NEISA', 'Providence': 'NEISA', 'Iowa State': 'MCSA', 'Iowa': 'MCSA', 'Indiana': 'MCSA', 'Davidson': 'SAISA', 'Oregon State': 'NWICSA', 'Western Washington': 'NWICSA', 'U. Rochester': 'MAISA', 'Army': 'MAISA', 'New Hampshire': 'NEISA', 'U. Connecticut': 'NEISA', 'UMass Dartmouth': 'NEISA', 'Wesleyan': 'NEISA', 'U. Mass/ Amherst': 'NEISA', 'U New England': 'NEISA', 'Denison': 'MCSA', 'Northern Michigan': 'MCSA', 'Ohio': 'MCSA', 'Pennsylvania': 'MAISA', 'Villanova': 'MAISA', 'Maine Maritime': 'NEISA', 'Michigan Tech': 'MCSA', 'Illinois': 'MCSA', 'Chicago': 'MCSA', 'Northwestern': 'MCSA', 'Grand Valley State': 'MCSA', 'Washington U': 'MCSA', 'Marquette': 'MCSA', 'Lake Forest': 'MCSA', 'Cornell': 'MAISA', 'Oregon': 'NWICSA', 'Portland State': 'NWICSA', 'Princeton': 'MAISA', "Queen's": 'MAISA', 'Penn State': 'MAISA', 'Ocean County': 'MAISA', 'Delaware': 'MAISA', 'Rutgers': 'MAISA', 'Worcester Polytech': 'NEISA', 'Emmanuel College': 'NEISA', "St. John's": 'MAISA', 'U Pittsburgh': 'MAISA', 'Webb Institute': 'MAISA', 'McGill': 'NEISA', 'Citadel': 'SAISA', 'Colgate': 'MAISA', 'Catholic U America': 'MAISA', 'Loyola College': 'MAISA', 'Ottawa': 'MAISA', 'Royal Military': 'MAISA', 'Dalhousie': 'NEISA', 'U Toronto': 'MAISA', 'New Orleans': 'SEISA', 'Kansas': 'SEISA', 'Bentley': 'NEISA', 'Brandeis': 'NEISA', 'Cal Maritime': 'PCCSC', 'San Diego State': 'PCCSC', 'Loyola': 'SEISA', 'North Texas': 'SEISA', 'Vanderbilt': 'SAISA', 'Purdue': 'MCSA', 'North Carolina': 'SAISA', 'Hillsdale': 'MCSA', 'Amherst': 'NEISA', 'Williams': 'NEISA', 'Hamilton': 'MAISA', 'Rochester': 'MAISA', 'Wellesley': 'NEISA', 'Hosei Univerisity': 'GUEST', 'Colorado': 'SEISA', 'John Carroll': 'MCSA', 'U.  Mass/ Boston': 'NEISA', 'Mercyhurst': 'MAISA', 'Penn State Behrend': 'MAISA', 'Indiana U Pennsylvan': 'MAISA', 'U Nebraska': 'MCSA', 'U Maine': 'NEISA', 'Texas Christian': 'SEISA', 'Embry-Riddle': 'SAISA', 'Palm Beach Atlantic': 'SAISA', 'U of Central Florida': 'SAISA', 'Baldwin-Wallace': 'MCSA', "Saint Mary's College": 'MCSA', 'Olin': 'NEISA', 'Baylor': 'SEISA', 'Texas Tech': 'SEISA', 'Wake Forest': 'SAISA', 'Georgia Southern': 'SAISA', 'East Carolina': 'SAISA', 'Florida Tech': 'SAISA', 'Saint Thomas': 'MCSA', 'Cincinnati': 'MCSA', 'Florida Gulf Coast': 'SAISA', 'Saginaw Valley': 'MCSA', 'Coastal Georgia': 'SAISA', 'Cleveland State': 'MCSA', 'Sewanee': 'SAISA', 'Case Western': 'MCSA', 'Oklahoma': 'SEISA', 'Gonzaga': 'PCCSC'}


In [171]:
people = {}

In [172]:
#create people
# people = {}
# Function to add a sailor to the dictionary
def add_sailor(group, seasons_group, years_group, links_group, role):
    global people
    for sailor, teams in group.items():
        if f"{sailor}/{role}" not in people.keys():
            # If no teams are associated, set "Unknown"
            teams = teams if len(teams) > 0 else ["Unknown"]

            # Retrieve the precomputed seasons
            seasons = seasons_group.get(sailor, [])
            year = years_group.get(sailor, [])
            link = links_group.get(sailor, [])
            
            # Add the sailor to the people dictionary
            people[f"{sailor}/{role}"] = Sailor(sailor, year, link, teams, role, list(seasons))
            
def setupPeople():
    try:
        df_s = pd.read_json("sailorsasf.json")
    except:
        df_s = pd.DataFrame(columns=['Sailor'])
        
    global people
    people = {}

    for sailor in list(df_s['Sailor'].unique()):
        # print(sailor)
        positions = df_s.loc[df_s['Sailor'] == sailor, 'Pos']
        for pos in positions:
            teams = df_s.loc[(df_s['Sailor'] == sailor)& (df_s['Pos'] == pos), 'Teams'].iat[0]
            seasons = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Seasons'].iat[0]
            year = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'GradYear'].iat[0]
            link = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Link'].iat[0]
            rating = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Elo'].iat[0]
            rank = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Rank'].iat[0]
            races = df_s.loc[(df_s['Sailor'] == sailor) & (df_s['Pos'] == pos), 'Races'].iat[0]
            people[sailor + "/" + pos] = Sailor(sailor, year, link, teams, pos, seasons, rank, rating, races)

    # Pre-group the data for skippers and crews
    skipper_groups = df_races_skipper.groupby('Sailor')['Team'].unique()
    crew_groups = df_races_crew.groupby('Sailor')['Team'].unique()

    # Precompute seasons for skippers and crew
    skipper_seasons = (
        df_races_skipper.assign(Season=df_races_skipper['raceID'].str.split('/').str[0])
        .groupby('Sailor')['Season']
        .unique()
    )

    crew_seasons = (
        df_races_crew.assign(Season=df_races_crew['raceID'].str.split('/').str[0])
        .groupby('Sailor')['Season']
        .unique()
    )

    skipper_years = (
        df_races_skipper.assign(Season=df_races_skipper['raceID'].str.split('/').str[0])
        .groupby('Sailor')['GradYear']
        .unique()
    )

    crew_years = (
        df_races_crew.assign(Season=df_races_crew['raceID'].str.split('/').str[0])
        .groupby('Sailor')['GradYear']
        .unique()
    )

    skipper_links = (
        df_races_skipper.assign(Season=df_races_skipper['raceID'].str.split('/').str[0])
        .groupby('Sailor')['Link']
        .unique()
    )

    crew_links = (
        df_races_crew.assign(Season=df_races_crew['raceID'].str.split('/').str[0])
        .groupby('Sailor')['Link']
        .unique()
    )

    # Add skippers and crew
    add_sailor(skipper_groups, skipper_seasons, skipper_years, skipper_links, 'Skipper')
    add_sailor(crew_groups, crew_seasons, crew_years, crew_links, 'Crew')

In [173]:
# %%scalene

# extras = False
people = {}
setupPeople()
residuals = []
leng = len(df_races['raceID'].unique())
# for type, df_races in zip(['/Skipper'], [df_races_skipper]):
for type, df_races in zip(['/Skipper', '/Crew'], [df_races_skipper, df_races_crew]):
    i = 0
    grouped = df_races.groupby(['Date', 'Regatta', 'adjusted_raceID', 'raceID'], sort=False)

    for (date, regatta, race, actualID), scores in grouped:
        #scores = scores.sort_values(by=['raceNum', 'raceDiv'])

        if i % 1000 == 0:
            print(f"Currently analyzing race {i}/{leng} Regatta:{regatta}, Date:{date}")
        i += 1

        sailors = scores['Sailor']
        scoreVals = list(scores['Score'])
        if sailors.empty:
            continue
        if np.isnan(scoreVals[0]): # B division did not complete the set
            continue
        
        racers = [people[p + type] for p in sailors]
        startingElos = [r.r.ordinal() for r in racers]
        
        regattaAvg = sum(startingElos) / len(racers)
        
        ratings = [[r.r] for r in racers]

        # Skip races with fewer than 2 participants
        if len(ratings) < 2:
            # print(regatta, "did not have enough sailors??")
            continue

        # Rate using the model
        ratings = model.rate(ratings, scoreVals) 

        predictions = model.predict_rank(ratings)
        
        for pred, score in zip(predictions, scoreVals):
            residuals.append(score - pred[0])

        # Update racers' ratings
        for racer, new_rating in zip(racers, ratings):
            racer.r = new_rating[0]

        # Calculate changes
        changes = [racers[i].r.ordinal() - startingElos[i] for i in range(len(racers))]
        partners = list(scores['Partner'])
        venue = scores['Venue'].iat[0]
        scoring = scores['Scoring'].iat[0]
        # teamRegions = {'MIT': "NEISA"}
        regions = [teamRegions[p.teams[-1]] if p.teams[-1] in teamRegions.keys() else None for p in racers]
        
        for sailor, score, pred, change, partner in zip(racers, scoreVals, predictions, changes, partners):
            crossRace = sum(1 for reg in regions if sailor.teams[-1] in teamRegions.keys() and reg != teamRegions[sailor.teams[-1]] and reg is not None)
            sailor.races.append({
                'score': int(score),
                'predicted': pred[0],
                'ratio': 1 - ((int(score) - 1) / (len(ratings) - 1)), 
                'change':change,
                'regAvg':regattaAvg,
                'cross': crossRace,
                # 'sailors': sailors,
                # 'teams': sailors['Teams'].apply(lambda x: x.teams[-1]).unique(),
                'newRating': sailor.r.ordinal(),
                'date' :date,
                'partner': partner,
                'venue': venue,
                'raceID': actualID,
                'scoring': scoring
            })
            
me = np.array(residuals).mean()
mse = (np.array(residuals) ** 2).mean()
print(me, mse)

Currently analyzing race 0/32569 Regatta:s16/peter-wenner-rainbow-invite, Date:2016-01-16 00:00:00
Currently analyzing race 1000/32569 Regatta:s16/2016-seisa-coed-dinghy, Date:2016-04-16 00:00:00
Currently analyzing race 2000/32569 Regatta:f16/fall-fury, Date:2016-09-10 00:00:00
Currently analyzing race 3000/32569 Regatta:f16/susan-rogers-75, Date:2016-10-01 00:00:00
Currently analyzing race 4000/32569 Regatta:s17/mustang-open, Date:2017-03-04 00:00:00
Currently analyzing race 5000/32569 Regatta:s17/navy-spring, Date:2017-04-15 00:00:00
Currently analyzing race 6000/32569 Regatta:f17/shu, Date:2017-09-16 00:00:00
Currently analyzing race 7000/32569 Regatta:f17/captain-hurst-bowl, Date:2017-10-14 00:00:00
Currently analyzing race 8000/32569 Regatta:s18/barnyard-bizarre, Date:2018-03-03 00:00:00
Currently analyzing race 9000/32569 Regatta:s18/owen-mosbacher-knapp-trophies, Date:2018-04-14 00:00:00
Currently analyzing race 10000/32569 Regatta:f18/norman-reid, Date:2018-09-22 00:00:00
Curr

In [174]:
# Compute season rivals
grouped = df_races_full[df_races_full['raceID'].str.startswith('f24')].groupby('adjusted_raceID')

i = 0
leng = len(grouped)
for raceID, scores in grouped:
    if i % 100 == 0:
        print(f"Currently analyzing race {i}/{leng} Regatta:{raceID}")
    i += 1
    
    sailor_scores = scores.set_index('Sailor')[['Position', 'Score', 'Team']]

    for sailor, sailor_data in sailor_scores.iterrows():
        pos = sailor_data['Position']
        score = sailor_data['Score']
        p = people[f"{sailor}/{pos}"]
        
        others = sailor_scores.iterrows()

        for other_sailor, other_sailor_data in others:
            if other_sailor != sailor:
                other_pos = other_sailor_data['Position']
                if pos == other_pos:
                    other_score = other_sailor_data['Score']
                    other_team = other_sailor_data['Team']
                    
                    if other_sailor not in p.rivals:
                        p.rivals[other_sailor] = {'races': 0, 'team': other_team, 'wins': 0}
                    
                    p.rivals[other_sailor]['races'] += 1
                    if other_score > score:
                        p.rivals[other_sailor]['wins'] += 1

Currently analyzing race 0/2261 Regatta:f24/2024-triangle-tango-intercollegiate/1A
Currently analyzing race 100/2261 Regatta:f24/boiler-cup/1A
Currently analyzing race 200/2261 Regatta:f24/cazenovia-fall-open/3A
Currently analyzing race 300/2261 Regatta:f24/chesapeake-open/1
Currently analyzing race 400/2261 Regatta:f24/dick-allsop/5A
Currently analyzing race 500/2261 Regatta:f24/fall-pacific-coast/5A
Currently analyzing race 600/2261 Regatta:f24/harry-anderson-jr/5A
Currently analyzing race 700/2261 Regatta:f24/jefferson-cup/1A
Currently analyzing race 800/2261 Regatta:f24/kingsmill-cup/3B
Currently analyzing race 900/2261 Regatta:f24/maisa-women-fall-dinghy/1B
Currently analyzing race 1000/2261 Regatta:f24/moody/1B
Currently analyzing race 1100/2261 Regatta:f24/nevins/7B
Currently analyzing race 1200/2261 Regatta:f24/open-atlantic-coast-final/3B
Currently analyzing race 1300/2261 Regatta:f24/open-sugar-bowl/12
Currently analyzing race 1400/2261 Regatta:f24/rebecca-becky-blank/1A
Curr

In [175]:
# Filter sailors who have 'f24' in their seasons list
eligible_sailors = [p for p in people.values() if 'f24' in p.seasons and sum([race['cross'] for race in p.races]) > 20]

for p in people.values():
    p.rank = 0

for pos in ["Skipper", "Crew"]:
    for i,s in enumerate(sorted([p for p in eligible_sailors if p.pos == pos], key=lambda p: p.r.ordinal(), reverse=True)):
        s.rank = i + 1

allRows = []
for p in list(people.values()):
    avgRatio = float(np.array([r['ratio'] for r in p.races]).mean())
    p.avgRatio = avgRatio
    allRows.append([p.name, p.year, p.link, p.rank, p.teams, p.pos, p.r.ordinal(), avgRatio, p.r.sigma, p.seasons, sum([race['cross'] for race in p.races]), p.races, p.rivals])
    
df_sailors = pd.DataFrame(allRows, columns=['Sailor','GradYear', 'Link', 'Rank', 'Teams', 'Pos', 'Elo','avgRatio','Sigma', 'Seasons','Cross', 'Races', 'Rivals'])

df_sailors.to_json('sailorsexperiment17.json', index=False)
df_sailors

/var/folders/10/cpsfwdm906d0_fmx9wrl17gh0000gn/T/ipykernel_27662/1930868614.py:13: RuntimeWarning: Mean of empty slice.
  avgRatio = float(np.array([r['ratio'] for r in p.races]).mean())
/Users/carter/.pyenv/versions/3.12.6/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Sailor,GradYear,Link,Rank,Teams,Pos,Elo,avgRatio,Sigma,Seasons,Cross,Races,Rivals
0,A. Tucker Atterbury,[16],[tucker-atterbury],0,[Roger Williams],Skipper,653.405876,0.513406,259.822739,"[s16, f16]",249,"[{'score': 1, 'predicted': 8, 'ratio': 1.0, 'c...",{}
1,A.J. Crane,[04 *],[None],0,[Tufts],Skipper,73.993374,0.488095,328.573697,[s23],0,"[{'score': 10, 'predicted': 12, 'ratio': 0.571...",{}
2,AJ Kozaritz,[23],[aj-kozaritz],0,[Rhode Island],Skipper,-54.201893,-0.500000,262.716030,[f22],0,"[{'score': 10, 'predicted': 8, 'ratio': 0.0, '...",{}
3,AJ McRitchie,[22],[aj-mcritchie],0,[Miami University],Skipper,-497.277569,0.033333,251.991312,[s22],0,"[{'score': 6, 'predicted': 5, 'ratio': 0.0, 'c...",{}
4,AJ Reiter,[17],[aj-reiter],0,[Georgetown],Skipper,1349.916015,0.569925,240.242279,"[f16, s17]",372,"[{'score': 15, 'predicted': 17, 'ratio': 0.263...",{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18208,victor lu,[19],[victor-lu],0,[Penn State],Crew,-401.107173,0.114286,291.642885,[f17],0,"[{'score': 11, 'predicted': 6, 'ratio': 0.0, '...",{}
18209,weston sanders,[20],[weston-sanders],0,[Clemson],Crew,-50.658884,0.288889,297.631311,[f17],5,"[{'score': 7, 'predicted': 5, 'ratio': 0.33333...",{}
18210,william caggiano,[27],[william-caggiano],0,[NY Maritime],Crew,140.202522,0.342857,290.364624,[f22],42,"[{'score': 8, 'predicted': 12, 'ratio': 0.5333...",{}
18211,ximena mendez,[20],[ximena-mendez],0,[Loyola College],Crew,187.909130,0.504167,281.368928,[s18],0,"[{'score': 6, 'predicted': 5, 'ratio': 0.16666...",{}


In [164]:
win_ratios = []

for rival, stats in people['Sabrina Anderson/Crew'].rivals.items():
    races = stats['races']
    wins = stats['wins']
    team = stats['team']
    win_ratio = wins / races if races > 0 else 0  # Handle division by zero
    win_ratios.append((rival, races, win_ratio, team))

# Sort the list by win_ratio in descending order
win_ratios_sorted = sorted(win_ratios, key=lambda x: x[1], reverse=True)

# Print the sorted win ratios
for rival, races, win_ratio, team in win_ratios_sorted:
    print(f"{rival}, Team: {team}, Races: {races}, Win Percentage: {win_ratio * 100:.0f}%")

Erin McDonagh, Team: Wisconsin, Races: 8, Win Percentage: 100%
Hope Campbell, Team: Cornell, Races: 2, Win Percentage: 100%
Cindy Wang, Team: Tufts, Races: 3, Win Percentage: 100%
Cody Lamoreux, Team: Tufts, Races: 3, Win Percentage: 100%
Sophia Mulvania, Team: Cornell, Races: 1, Win Percentage: 100%
Henry Tindall, Team: Tufts, Races: 1, Win Percentage: 100%
Kate Moran, Team: Cornell, Races: 2, Win Percentage: 100%
Helen Horangic, Team: Brown, Races: 1, Win Percentage: 100%
Samantha Karlson, Team: MIT, Races: 9, Win Percentage: 100%
Alejandro Amezcua, Team: Connecticut College, Races: 3, Win Percentage: 100%
Lily Schwartz, Team: Eckerd, Races: 4, Win Percentage: 100%
Avalon Everett, Team: UC Santa Barbara, Races: 13, Win Percentage: 100%
Caitlin Costello, Team: Wisconsin, Races: 13, Win Percentage: 100%
Sarah Krajewski, Team: Coast Guard, Races: 1, Win Percentage: 100%
Ella Adelman, Team: George Washington, Races: 6, Win Percentage: 100%
Avery Guck, Team: Eckerd, Races: 9, Win Percenta

In [ ]:
# Updates only changed sailors
col = db.collection('sailorsElo')

for doc in col.limit(10).stream():
    doc_id = doc.id
    data = doc.to_dict()
    print(doc_id)
    changes = {}
    person = people[data['Name'] + "/" + data['Position']]
    if len(person.races) < 1:
        print("no races found for sailor, skipping...")
        continue
    
    print(data['Name'])
    try:
        data['Link']
    except:
        changes['Link'] = list(person.link)[0]
        # print("No link!")
    try:
        data['Year']
    except:
        changes['Year'] = int(list(person.year)[0][:2])
        # print("No year!")
    
    # print(person.races[0])
    # check races
    changes['races'] = firestore.ArrayUnion(person.races)
    # for raceID in [r['raceID'] for r in person.races]:
    #     if raceID not in [r['raceID'] for r in data['races']]:
    #         changes['races'].append(raceID)
    #         print("needs update!", data['races'])
    #         print(person.races)
    print(changes)
    col.document(doc_id).update(changes)

00DrKGRiyUzy6fPhwAUW


KeyError: 'Name'

In [176]:
# Initialize Firestore client
col = db.collection('sailorsElo')

# Initialize the batch
batch = db.batch()

# Number of documents to commit in each batch
batch_size = 20

# Iterate over the people values
for i, p in enumerate(people.values()):
    # Prepare the document data to be written
    doc_data = {
        "Name": p.name,
        "Position": p.pos,
        "Teams": list(p.teams),
        "Rating": int(p.r.mu),
        "GlobalRank": int(p.rank),
        "Link": list(p.link)[0],
        "Year": int(list(p.year)[0][:2]),
        "races": [{'sailor': p.name,
                'pos': p.pos,
                "raceID": race['raceID'], 
                "score": float(race['score']), 
                "predicted": int(race['predicted']), 
                "change": float(race['change']), 
                'regAvg': float(race['regAvg']), 
                'newRating': float(race['newRating']),
                'date': race['date'],
                'partner': race['partner'],
                'ratio': float(race['ratio']),
                'venue': race['venue']
                } for race in p.races],
        "Rivals": p.rivals
    }
    
    # Add the set operation to the batch
    doc_ref = col.document()  # This creates a new document with an auto-generated ID
    batch.set(doc_ref, doc_data)
    
    # Commit the batch every 20 documents
    if (i + 1) % batch_size == 0:
        batch.commit()
        batch = db.batch()  # Start a new batch for the next set of documents

# Commit any remaining operations if there are less than 20 documents left
if (i + 1) % batch_size != 0:
    batch.commit()


In [187]:
%%scalene
import requests
from bs4 import BeautifulSoup
from collections import defaultdict

def getCounts(races):
    season_counts = defaultdict(int)
    
    for race in races:
        season = race["raceID"].split("/")[0]
        season_counts[season] += 1

    return dict(season_counts)

# batch = db.batch()
col = db.collection('eloTeams')
lenteams = len(list(df_races['Team'].unique()))
teams = []
bySailors = df_races_full.groupby('Sailor')

for i,team in enumerate(list(df_races['Team'].unique())):
    print(f"{i}/{lenteams} {team}")
    avg = df_sailors.loc[df_sailors['Teams'].apply(lambda x: team in x), 'Elo'].mean()
    avgRatio = df_sailors.loc[df_sailors['Teams'].apply(lambda x: team in x), 'avgRatio'].mean()
    numCurMembers = len(df_sailors.loc[(df_sailors['Teams'].apply(lambda x: team in x)) & (df_sailors['Seasons'].apply(lambda x: 'f24' in x))])
    
    # teamLink = df_races.loc[df_races['Team'] == team, 'Teamlink'].iloc[0]
    # url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"
    # # print(url)
    # page = requests.get(url)
    # teamPage = BeautifulSoup(page.content, 'html.parser')
    
    # try:
    #     region = teamPage.find('span', class_="page-info-value").contents[0].contents[0]
    # except:
    #     print(url)
    #     continue
    
    members = [{"name": p.name, 
                'pos':p.pos,
                'teams': list(p.teams),
                'rating': int(p.r.mu),
                'avgRatio': float(p.avgRatio),
                'raceCount': getCounts(p.races),
                'seasons': p.seasons,
                'cross': sum([race['cross'] for race in p.races]),
                # 'crewSeasons': p.seasons if p.pos == 'Crew' else [],
                # 'skipperSeasons':list(df_races_full.loc[(df_races_full['Sailor'] == p.name) & (df_races_full['Position'] == 'Skipper')]['raceID'].str.split('/').str[0].unique()), 
                # 'crewSeasons':list(df_races_full.loc[(df_races_full['Sailor'] == p.name) & (df_races_full['Position'] == 'Crew')]['raceID'].str.split('/').str[0].unique()), 
                
                'globalrank': int(p.rank)} for p in people.values() if team in p.teams]
    
    teamRating = 0
    if numCurMembers > 0:
        teamRating = sum([p['rating'] * (p['raceCount']['f24']/ 5) for p in members if 'f24' in p['raceCount'].keys() and p['raceCount']['f24'] > 5]) / numCurMembers
    
    # print(members[0])
    topRating = 0
    if numCurMembers > 0:
        topSkippers = sum(sorted([p['rating'] for p in members
                                  if p['pos'] == 'Skipper' 
                                  and p['cross'] > 20
                                  and 'f24' in p['raceCount'].keys() 
                                  and p['raceCount']['f24'] > 5], reverse=True)[:4])
        
        topCrews = sum(sorted([p['rating']for p in members
                               if p['pos'] == 'Crew' 
                               and p['cross'] > 20
                               and 'f24' in p['raceCount'].keys() 
                               and p['raceCount']['f24'] > 5], reverse=True)[:4])
        topRating = (topSkippers + topCrews) / 8
    
    teams.append({"name":team, "avg": avg, 'avgRatio': avgRatio, 'teamRating': teamRating, 'topRating': topRating,  "region": region, "link": url, 'memberCount': numCurMembers})
    # print(topRating)
    # col.document().set({"name":team, "avg": avg, 'avgRatio': avgRatio,"region": region, "link": url, 'members': members, 'teamRating': teamRating})
    # if i > 20:    
    #     break
#     if i % 20 == 0: # commit every 20 documents
#             batch.commit()
# batch.commit()
# doc = db.collection('vars').document('eloTeams').set({"teams": teams})
teams

0/216 Hawaii
1/216 Brown
2/216 Southern Cal
3/216 Salve Regina
4/216 UC Santa Barbara
5/216 Cal Poly
6/216 Washington
7/216 Channel Islands
8/216 UC San Diego
9/216 British Columbia
10/216 UC Los Angeles
11/216 Westmont College
12/216 Arizona State
13/216 Texas A&M Galveston
14/216 Texas A&M
15/216 Tulane
16/216 Rice
17/216 Texas
18/216 Oklahoma State
19/216 Texas A&M C. Christ
20/216 Central Oklahoma
21/216 Notre Dame
22/216 Jacksonville
23/216 Florida
24/216 Tennessee
25/216 Rollins
26/216 North Carolina State
27/216 Georgia Tech
28/216 Auburn
29/216 Charleston
30/216 South Florida
31/216 Old Dominion
32/216 Eckerd
33/216 Florida State
34/216 U. Miami
35/216 UW Milwaukee
36/216 Stony Brook
37/216 Duke
38/216 Clemson
39/216 U South Carolina
40/216 UNC Wilmington
41/216 Georgia
42/216 Berkeley
43/216 CSU Long Beach
44/216 Monterey Bay
45/216 UC Irvine
46/216 UC Davis
47/216 Rhode Island
48/216 Georgetown
49/216 Dartmouth
50/216 MIT
51/216 George Washington
52/216 Navy
53/216 Fordham
54

In [71]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
import time

# Initialize a defaultdict to count races per season

def getCounts(races):
    # season_counts = {}
    season_counts = defaultdict(int)
    
    for race in races:
        season = race["raceID"].split("/")[0]
        season_counts[season] += 1

    return dict(season_counts)


# batch = db.batch()
# col = db.collection('eloTeams')
lenteams = len(list(df_races['Team'].unique()))
teams = {}
byTeam = df_races_full.groupby('Team')

for i,team in enumerate(list(df_races['Team'].unique())):
    thisTeam = df_sailors.loc[df_sailors['Teams'].apply(lambda x: team in x)]
    print(f"{i}/{lenteams} {team}")
    avg = thisTeam['Elo'].mean()
    avgRatio = thisTeam['avgRatio'].mean()
    numCurMembers = len(thisTeam.loc[df_sailors['Seasons'].apply(lambda x: 'f24' in x)])
    
    teamLink = byTeam.get_group(team)['Teamlink'].iat[0]
    url = f"https://scores.collegesailing.org/schools/{teamLink.split("/")[2]}"
    page = requests.get(url)
    teamPage = BeautifulSoup(page.content, 'html.parser')
    
    try:
        region = teamPage.find('span', class_="page-info-value").contents[0].contents[0]
    except:
        print(url)
        continue
    
    # members = [{"name": p.name,
    #             'pos':p.pos,
    #             'teams': list(p.teams),
    #             'rating': int(p.r.mu),
    #             'avgRatio': float(p.avgRatio),
    #             'raceCount': getCounts(p.races),
    #             'skipperSeasons':list(df_races_full.loc[(df_races_full['Sailor'] == p.name) & (df_races_full['Position'] == 'Skipper')]['raceID'].str.split('/').str[0].unique()), 
    #             'crewSeasons':list(df_races_full.loc[(df_races_full['Sailor'] == p.name) & (df_races_full['Position'] == 'Crew')]['raceID'].str.split('/').str[0].unique()),
    #             'globalrank': int(p.rank)} for p in people.values() if team in p.teams]
    
    # teamRating = sum([p['rating'] * (p['raceCount']['f24']/ 10) for p in members if 'f24' in p['raceCount'].keys()]) / numCurMembers
    # teams.append({"name":team, "avg": avg, 'avgRatio': avgRatio,"region": region, "link": url, 'memberCount': numCurMembers})
    teams[team] = region
    # print(teamRating)
    # col.document().set({"name":team, "avg": avg,"region": region, "link": url, 'members': members})
#     if i % 20 == 0: # commit every 20 documents
#             batch.commit()
# batch.commit()
# doc = db.collection('vars').document('eloTeams').set({"teams": teams})
print(teams)
df = pd.DataFrame(teams)
df.to_json('teamRegions.json')

0/216 Hawaii
1/216 Brown
2/216 Southern Cal
3/216 Salve Regina
4/216 UC Santa Barbara
5/216 Cal Poly
6/216 Washington
7/216 Channel Islands
8/216 UC San Diego
9/216 British Columbia
10/216 UC Los Angeles
11/216 Westmont College
12/216 Arizona State
13/216 Texas A&M Galveston
14/216 Texas A&M
15/216 Tulane
16/216 Rice
17/216 Texas
18/216 Oklahoma State
19/216 Texas A&M C. Christ
20/216 Central Oklahoma
21/216 Notre Dame
22/216 Jacksonville
23/216 Florida
24/216 Tennessee
25/216 Rollins
26/216 North Carolina State
27/216 Georgia Tech
28/216 Auburn
29/216 Charleston
30/216 South Florida
31/216 Old Dominion
32/216 Eckerd
33/216 Florida State
34/216 U. Miami
35/216 UW Milwaukee
36/216 Stony Brook
37/216 Duke
38/216 Clemson
39/216 U South Carolina
40/216 UNC Wilmington
41/216 Georgia
42/216 Berkeley
43/216 CSU Long Beach
44/216 Monterey Bay
45/216 UC Irvine
46/216 UC Davis
47/216 Rhode Island
48/216 Georgetown
49/216 Dartmouth
50/216 MIT
51/216 George Washington
52/216 Navy
53/216 Fordham
54

ValueError: If using all scalar values, you must pass an index

In [ ]:
byTeams = df_races_full.groupby('Team')
for team in list(byTeams)[:1]:
    print(team)

In [179]:
topSkippers = []
for p in sorted([p for p in people.values() if p.rank <= 100 and p.rank != 0 and p.pos == 'Skipper'],key=lambda p: p.rank):
    topSkippers.append({'name': p.name, 'rank': int(p.rank), 'pos': p.pos, 'team': list(p.teams), 'rating': p.r.mu, 'seasons': list(p.seasons)})
doc = db.collection('vars').document('topSkippers').set({"sailors": topSkippers})
topCrews = []
for p in sorted([p for p in people.values() if p.rank <= 100 and p.rank != 0 and p.pos == 'Crew'],key=lambda p: p.rank):
    topCrews.append({'name': p.name, 'rank': int(p.rank), 'pos': p.pos, 'team': list(p.teams), 'rating': p.r.mu, 'seasons': list(p.seasons)})
doc = db.collection('vars').document('topCrews').set({"sailors": topCrews})

In [52]:
import json
flattened_dict = {p.name: {'team': p.teams[-1], 'year': p.year[-1]} for p in people.values() if 'f24' in p.seasons}
# print(flattened_dict)
doc = db.collection('vars').document('allSailors').set({'allSailors': json.dumps(flattened_dict, separators=(',', ':'))})

In [51]:
print(list(df_races_full.loc[df_races_full['Sailor']=='Elliott Chalcraft', 'raceID']))

['f23/toni-deutsch-58/1B', 'f23/toni-deutsch-58/2B', 'f23/toni-deutsch-58/3B', 'f23/toni-deutsch-58/4B', 'f23/regis/1A', 'f23/regis/2A', 'f23/regis/3A', 'f23/regis/4A', 'f23/regis/5A', 'f23/regis/6A', 'f23/regis/7A', 'f23/regis/8A', 'f23/regis/9A', 'f23/hood/1A', 'f23/hood/2A', 'f23/hood/3A', 'f23/hood/4A', 'f23/hood/5A', 'f23/hood/6A', 'f23/hood/7A', 'f23/hood/8A', 'f23/hood/9A', 'f23/hood/10A', 'f23/hood/11A', 'f23/hood/12A', 'f23/hood/13A', 'f23/hood/14A', 'f23/george-warren-smith/1B', 'f23/george-warren-smith/2B', 'f23/george-warren-smith/3B', 'f23/george-warren-smith/4B', 'f23/george-warren-smith/5B', 'f23/george-warren-smith/6B', 'f23/george-warren-smith/7B', 'f23/george-warren-smith/8B', 'f23/george-warren-smith/9B', 'f23/stu-nelson/1C', 'f23/stu-nelson/2C', 'f23/stu-nelson/3C', 'f23/stu-nelson/4C', 'f23/stu-nelson/5C', 'f23/stu-nelson/6C', 'f23/stu-nelson/7C', 'f23/stu-nelson/8C', 'f23/stu-nelson/9C', 'f23/stu-nelson/10C', 'f23/stu-nelson/11C', 'f23/stu-nelson/12C', 'f23/savin-

In [50]:
len(list(df_races_full['Regatta'].unique()))

1723

In [ ]:
for regatta in list(df_races_full['Regatta'].unique()):
    races = df_races_full[df_races_full['Regatta'] == regatta]
    raceIDs = list(races['raceID'].unique())
    sailors = races['Sailor'].unique()
    
    racePpl = [{"Name":p.name, 
         "Position": p.pos,
         "Teams": list(p.teams),
         "Rating": int(p.r.mu),
         "GlobalRank": int(p.rank),
         "races": [{'sailor': p.name,
                    'pos': p.pos,
                    "raceID": race['raceID'], 
                    "score": float(race['score']), 
                    "predicted": int(race['predicted']), 
                    "change": float(race['change']), 
                    'regAvg': float(race['regAvg']), 
                    'newRating': float(race['newRating']),
                    'date': race['date'],
                    'partner':race['partner'],
                    'ratio': float(race['ratio']),
                    'venue': race['venue']
                    } for race in p.races if race['raceID'].split("/")[0] + "/" +race['raceID'].split("/")[1] == regatta]
        } for p in people.values() if p.name in sailors]
    
    # race = {'raceID':'', 'raceNum':0, 'div': '', 'sailors':[]}
    # person = {'name':'', 'rating':0, 'change':0, 'team': '', 'pos': '', 'div':'', 'partner': ''}
    
    # for race in races['raceID'].unique():
    #     sailors = races[races['raceID'] == race, 'Sailor'].unique()
    # for p in [p for p in people.values() if p.name in sailors]:
        # racePpl.append({'name':p.name, 'rating':p.rating, 'changes':p.changes, 'team': p.team, 'pos': '', 'div':'', 'partner': ''})
    # print(regatta)
    db.collection('eloRegattas').document().set({'regattaName': regatta,'raceIDs':raceIDs, 'sailors': racePpl}, timeout=15)

In [ ]:
def delete_collection(coll_ref, batch_size):
    if batch_size == 0:
        return

    docs = coll_ref.list_documents(page_size=batch_size)
    deleted = 0

    for doc in docs:
        if deleted % 50 == 0:
            print(f"{deleted} Deleting doc {doc.id} => {doc.get().to_dict()}")
        doc.delete()
        deleted = deleted + 1

    if deleted >= batch_size:
        return delete_collection(coll_ref, batch_size)
col = db.collection('eloRegattas')
delete_collection(col, 400)

In [7]:
df_elo.to_csv("elo19.csv",index=False)